In [1]:
import hashlib
import secrets

# 1. Public Parameters (Simplified for the demo)
# In production, use standard large primes (e.g., from RFC 3526)
p = 2**255 - 19  # A large prime
g = 2            # Generator

def generate_proof(secret_x):
    """Prover: Generate a proof of knowledge for secret_x."""
    # Step 1: Commitment (t = g^k mod p)
    # k is a one-time random 'nonce'
    k = secrets.randbelow(p)
    t = pow(g, k, p)
    
    # Step 2: Challenge (c)
    # In 'Non-Interactive' ZKP (Fiat-Shamir), we hash the commitment to get c
    # This removes the need for the Verifier to send a message back.
    c_hash = hashlib.sha256(str(t).encode()).hexdigest()
    c = int(c_hash, 16) % p
    
    # Step 3: Response (s = k + c*x)
    s = (k + c * secret_x) % (p - 1)
    
    return t, s

def verify_proof(public_y, t, s):
    """Verifier: Verify the proof without knowing secret_x."""
    # Reconstruct the challenge
    c_hash = hashlib.sha256(str(t).encode()).hexdigest()
    c = int(c_hash, 16) % p
    
    # Check if g^s == t * y^c (mod p)
    left_side = pow(g, s, p)
    right_side = (t * pow(public_y, c, p)) % p
    
    return left_side == right_side

# --- TEST RUN ---
secret_password = 123456789  # The 'Witness' (private)
public_key = pow(g, secret_password, p)  # The 'Statement' (public)

# Prover generates the proof
t, s = generate_proof(secret_password)
print(f"Proof generated.\nCommitment: {t}\nResponse: {s}")

# Verifier checks the proof
is_valid = verify_proof(public_key, t, s)
print(f"\nVerification Result: {'✅ SUCCESS' if is_valid else '❌ FAILED'}")

Proof generated.
Commitment: 10484266763605692677657377208460298558265203723181615407010341273859829493711
Response: 57232773224426153084663127649862969586652826416574927909436024920929536819592

Verification Result: ✅ SUCCESS
